<a href="https://colab.research.google.com/github/aysucengiz/ceng463-hw2/blob/task2/task2/CENG463_hw2_task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 2

Given a parliamentary speech in one of several languages, identify the ideology of the speaker’s party. In other words, this involves performing binary classification to determine whether the speaker’s party leans left (0) or right (1).

- Select only one country (excluding the UK) and use the parliamentary debates from that country to complete the assigned tasks.

  - **Chosen Country:** Ukraine

- Fine-tune a multilingual masked language model:

  - **Chosen Model:** XLMRoberta-base

- In addition, you are required to experiment with a multilingual causal language model:

  - **Chosen Model:** distilgpt2 (note: though gpt2 is a primarily English Model, it can handle multilingual data)

https://sidih.si/cdn/121/index.html

## Prepare the Data

Steps:
- Download
- Read with pandas
- Split


### Download and Read w/ Pandas

Download from the given lin in the homework pdf and unzip

In [1]:
!wget -O trainingset-ideology-power.zip https://zenodo.org/records/10450641/files/trainingset-ideology-power.zip?download=1

--2025-01-01 12:02:54--  https://zenodo.org/records/10450641/files/trainingset-ideology-power.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.43.25, 188.185.45.92, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.43.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 813899321 (776M) [application/octet-stream]
Saving to: ‘trainingset-ideology-power.zip’

trainingset-ideolog 100%[===================>] 776.19M  11.8MB/s    in 69s     

2025-01-01 12:04:04 (11.2 MB/s) - ‘trainingset-ideology-power.zip’ saved [813899321/813899321]



In [2]:
!unzip trainingset-ideology-power.zip -d trainingset-ideology-power

Archive:  trainingset-ideology-power.zip
   creating: trainingset-ideology-power/orientation/
  inflating: trainingset-ideology-power/orientation/orientation-at-train.tsv  
  inflating: trainingset-ideology-power/orientation/orientation-ba-train.tsv  
  inflating: trainingset-ideology-power/orientation/orientation-be-train.tsv  
  inflating: trainingset-ideology-power/orientation/orientation-bg-train.tsv  
  inflating: trainingset-ideology-power/orientation/orientation-cz-train.tsv  
  inflating: trainingset-ideology-power/orientation/orientation-dk-train.tsv  
  inflating: trainingset-ideology-power/orientation/orientation-ee-train.tsv  
  inflating: trainingset-ideology-power/orientation/orientation-es-ct-train.tsv  
  inflating: trainingset-ideology-power/orientation/orientation-es-ga-train.tsv  
  inflating: trainingset-ideology-power/orientation/orientation-es-train.tsv  
  inflating: trainingset-ideology-power/orientation/orientation-fi-train.tsv  
  inflating: trainingset-ideolo

Read with pandas and print the head

In [3]:
import pandas as pd
from tabulate import tabulate

data_df = pd.read_csv("./trainingset-ideology-power/power/power-tr-train.tsv", delimiter="\t")
print(data_df.head())

        id                           speaker sex  \
0  tr18146  ca2031caa4032c51980160359953d507   M   
1  tr18147  4cee0addb3c69f6866869b180f90d45f   M   
2  tr18148  b3d7f76d74ec268492f8190ca123a6b2   M   
3  tr18149  722efac7138c8197a9d1e97eed3a8b18   M   
4  tr18150  fcc61122f3553c57ae207adeb1a1af84   M   

                                                text  \
0  Yeni yasama döneminin ülkemiz için, milletimiz...   
1  Sayın Başkan, değerli milletvekilleri; bugün, ...   
2  Sayın Başkanım, öncelikle yüce Meclisin Başkan...   
3  24’üncü Dönem Meclis Başkanlığına seçilmenizde...   
4  Usul tartışmasında 2 kişi lehte 2 kişi aleyhte...   

                                             text_en  label  
0  Mr. President, dear lawmakers, I salute you, a...      0  
1  Mr. President, members of lawmakers, as I spea...      0  
2  Mr. President, I'm here to share with you the ...      0  
3  Mr. President, under the principles determined...      0  
4  Two in favour of two in the legal deb

### Split

If you intend to participate in the shared task, you can evaluate your model on the test set.
Since the test dataset lacks labels, you must split your training data into **90% for training** and
**10% for testing**. Ensure the split is performed in a `stratified` manner to maintain the proportion
of labels in both subsets.

[Split guide](https://scikit-learn.org/1.5/modules/generated/sklearn.model_selection.train_test_split.html) from the pdf.

[Stratify guide](https://stackoverflow.com/questions/34842405/parameter-stratify-from-method-train-test-split-scikit-learn) from the pdf.

In [4]:
import sklearn.model_selection

train_df, test_df = sklearn.model_selection.train_test_split(data_df, test_size=0.1, stratify=data_df["label"])

print("Train data proportion of label: \n",train_df['label'].value_counts(normalize=True) * 100)
print("Test data proportion of label: \n",test_df['label'].value_counts(normalize=True) * 100)

Train data proportion of label: 
 label
1    51.377437
0    48.622563
Name: proportion, dtype: float64
Test data proportion of label: 
 label
1    51.408856
0    48.591144
Name: proportion, dtype: float64


## Masked LM

Fine-tune the selected masked language model for each task: For one task use ”text_en” and for the other task use ”text” (original language).

- **Chosen language for this task:** ”text_en” (English)


In [5]:
!pip install transformers datasets evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


### Tokenize the data

In [6]:
from transformers import XLMRobertaTokenizer

roberta_tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

In [7]:
def tokenize_it(text):
    return roberta_tokenizer(text, padding="max_length", truncation=True)

train_tokenized = tokenize_it(train_df["text_en"].tolist())
test_tokenized = tokenize_it(test_df["text_en"].tolist())

### Fine-Tune the Model

Create the dataloader of the datasets.

In [8]:
import torch
from torch.utils.data import TensorDataset, DataLoader

def data_loader(data,tokenized):
  # Convert tokenized data to tensors
  input_ids = torch.tensor(tokenized["input_ids"])
  attention_mask = torch.tensor(tokenized["attention_mask"])
  labels = torch.tensor(data["label"].tolist(), dtype=torch.long)

  # Create TensorDataset
  dataset = TensorDataset(input_ids, attention_mask, labels)

  # Create DataLoader
  data_loader_result = DataLoader(dataset, batch_size=32, shuffle=True)
  return data_loader_result

train_dataloader = data_loader(train_df,train_tokenized)
test_dataloader = data_loader(test_df,test_tokenized)

In [9]:
from transformers import XLMRobertaForSequenceClassification, AdamW

model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=2)
optimizer = AdamW(model.parameters(), lr=5e-5)

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Set some variables for the training.

In [10]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

Determine the device.

In [11]:
import torch
from accelerate.test_utils.testing import get_backend

device, _, _ = get_backend() # automatically detects the underlying device type (CUDA, CPU, XPU, MPS, etc.)
model.to(device)

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=

Start training

In [12]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        input_ids, attention_mask, labels = batch
        batch = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        progress_bar.set_postfix({"loss": loss.item()})

  0%|          | 0/1467 [00:00<?, ?it/s]

Evaluate the model trained

In [1]:
import evaluate

metric = evaluate.load("accuracy")
model.eval()
for batch in test_dataloader:
    input_ids, attention_mask, labels = batch
    batch = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
            }
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

ModuleNotFoundError: No module named 'evaluate'

# Causal LM

For each task, perform inference using the selected causal language model twice: Once using ”text_en” and once using ”text” (original language).

Since distilgpt is a primarily English model, I wanted to see the difference training would make. Therefore, I did 4 different scenarios instead of 2.

- Trained original
- Trained English
- Untrained original
- Untrained English

In [ ]:
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


Load the tokenizer

In [ ]:
from transformers import AutoTokenizer

model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Add a padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token


Load the models

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 2

model_train_en = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
model_train_en.config.pad_token_id = tokenizer.pad_token_id

model_en = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
model_en.config.pad_token_id = tokenizer.pad_token_id

model_train_orig = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
model_train_orig.config.pad_token_id = tokenizer.pad_token_id

model_orig = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
model_orig.config.pad_token_id = tokenizer.pad_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to b

## Training

Start with tokenizing

In [ ]:
def preprocess_text(data):
    return tokenizer(data["text"], truncation=True, padding="max_length", max_length=128)
def preprocess_text_en(data):
    return tokenizer(data["text_en"], truncation=True, padding="max_length", max_length=128)


tokenized_train_en = train_df.apply(preprocess_text_en, axis=1).values  # Get the values as a list of dictionaries
tokenized_test_en = test_df.apply(preprocess_text_en, axis=1).values

tokenized_train_orig = train_df.apply(preprocess_text, axis=1).values  # Get the values as a list of dictionaries
tokenized_test_orig = test_df.apply(preprocess_text, axis=1).values

Create the dataloader of the datasets

In [ ]:
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm
import torch


# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


# train
train_dataset_en = CustomDataset({k: [d[k] for d in tokenized_train_en] for k in tokenized_train_en[0]}, train_df["label"].tolist())
train_dataloader_en = DataLoader(train_dataset_en, batch_size=16)

# test
test_dataset_en = CustomDataset({k: [d[k] for d in tokenized_test_en] for k in tokenized_test_en[0]}, test_df["label"].tolist())
test_dataloader_en = DataLoader(test_dataset_en, batch_size=16)


# train
train_dataset_orig = CustomDataset({k: [d[k] for d in tokenized_train_orig] for k in tokenized_train_orig[0]}, train_df["label"].tolist())
train_dataloader_orig = DataLoader(train_dataset_orig, batch_size=16)

# test
test_dataset_orig = CustomDataset({k: [d[k] for d in tokenized_test_orig] for k in tokenized_test_orig[0]}, test_df["label"].tolist())
test_dataloader_orig = DataLoader(test_dataset_orig, batch_size=16)


### Training- English



In [ ]:
import torch
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_train_en.to(device)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)

In [ ]:
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm.auto import tqdm
import torch

# Define optimizer and scheduler
optimizer = AdamW(model_train_en.parameters(), lr=5e-5)

# Total number of training steps
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader_en)

# Learning rate scheduler
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)


loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
for epoch in range(num_epochs):
    model_train_en.train()
    loop = tqdm(train_dataloader_en, desc=f"Epoch {epoch}")

    for batch in loop:
        # Move batch to device
        input_ids = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)

        # Forward pass
        outputs = model_train_en(input_ids=input_ids, labels=labels)
        loss = outputs.loss  # GPT-2 provides loss when `labels` are passed

        # Backward pass and optimization step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update learning rate scheduler
        lr_scheduler.step()

        # Update progress bar
        loop.set_postfix(loss=loss.item())


Epoch 0:   0%|          | 0/908 [00:00<?, ?it/s]

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


Epoch 1:   0%|          | 0/908 [00:00<?, ?it/s]

Epoch 2:   0%|          | 0/908 [00:00<?, ?it/s]

### Training- Original Language

* Original Language - Trained

In [ ]:
import torch
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_train_orig.to(device)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)

In [ ]:
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm.auto import tqdm
import torch

# Define optimizer and scheduler
optimizer = AdamW(model_train_orig.parameters(), lr=5e-5)

# Total number of training steps
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader_orig)

# Learning rate scheduler
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

# Loss function for classification (built into GPT-style models when `labels` are passed)
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
for epoch in range(num_epochs):
    model_train_orig.train()
    loop = tqdm(train_dataloader_orig, desc=f"Epoch {epoch}")

    for batch in loop:
        # Move batch to device
        input_ids = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)

        # Forward pass
        outputs = model_train_orig(input_ids=input_ids, labels=labels)
        loss = outputs.loss  # GPT-2 provides loss when `labels` are passed

        # Backward pass and optimization step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update learning rate scheduler
        lr_scheduler.step()

        # Update progress bar
        loop.set_postfix(loss=loss.item())


Epoch 0:   0%|          | 0/908 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/908 [00:00<?, ?it/s]

Epoch 2:   0%|          | 0/908 [00:00<?, ?it/s]

## Testing



* Original Language - Untrained

In [ ]:
from datasets import Dataset
from sklearn.metrics import accuracy_score
from transformers import pipeline

# Convert the test DataFrame to a Hugging Face Dataset
test_dataset = Dataset.from_pandas(test_df)

# Prepare the classifier pipeline
classifier = pipeline("text-classification", model=model_orig, tokenizer=tokenizer, device=0)

# Define a function for prediction
def predict(batch):
    results = classifier(batch["text"], truncation=True, max_length=128)
    predicted_labels = [int(result["label"].split("_")[-1]) for result in results]
    return {"predicted_label": predicted_labels}

# Apply the pipeline to the dataset with batching
predictions = test_dataset.map(predict, batched=True, batch_size=32)

# Compute accuracy
true_labels = test_dataset["label"]
predicted_labels = predictions["predicted_label"]

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")


Device set to use cuda:0


Map:   0%|          | 0/1614 [00:00<?, ? examples/s]

Accuracy: 42.94%


* Original Language - Trained

In [ ]:
from datasets import Dataset
from sklearn.metrics import accuracy_score
from transformers import pipeline

# Convert the test DataFrame to a Hugging Face Dataset
test_dataset = Dataset.from_pandas(test_df)

# Prepare the classifier pipeline
classifier = pipeline("text-classification", model=model_train_orig, tokenizer=tokenizer, device=0)

# Define a function for prediction
def predict(batch):
    results = classifier(batch["text"], truncation=True, max_length=128)
    predicted_labels = [int(result["label"].split("_")[-1]) for result in results]
    return {"predicted_label": predicted_labels}

# Apply the pipeline to the dataset with batching
predictions = test_dataset.map(predict, batched=True, batch_size=32)

# Compute accuracy
true_labels = test_dataset["label"]
predicted_labels = predictions["predicted_label"]

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")


Device set to use cuda:0


Map:   0%|          | 0/1614 [00:00<?, ? examples/s]

Accuracy: 66.48%


* English - Untrained

In [ ]:
from datasets import Dataset
from sklearn.metrics import accuracy_score
from transformers import pipeline

# Convert the test DataFrame to a Hugging Face Dataset
test_dataset = Dataset.from_pandas(test_df)

# Prepare the classifier pipeline
classifier = pipeline("text-classification", model=model_en, tokenizer=tokenizer, device=0)

# Define a function for prediction
def predict(batch):
    results = classifier(batch["text_en"], truncation=True, max_length=128)
    predicted_labels = [int(result["label"].split("_")[-1]) for result in results]
    return {"predicted_label": predicted_labels}

# Apply the pipeline to the dataset with batching
predictions = test_dataset.map(predict, batched=True, batch_size=32)

# Compute accuracy
true_labels = test_dataset["label"]
predicted_labels = predictions["predicted_label"]

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")


Device set to use cuda:0


Map:   0%|          | 0/1614 [00:00<?, ? examples/s]

Accuracy: 58.12%


* English - Trained

In [ ]:
from datasets import Dataset
from sklearn.metrics import accuracy_score
from transformers import pipeline

# Convert the test DataFrame to Dataset
test_dataset = Dataset.from_pandas(test_df)

# pipeline
classifier = pipeline("text-classification", model=model_train_en, tokenizer=tokenizer, device=0)


def predict(batch):
    results = classifier(batch["text_en"], truncation=True, max_length=128)
    predicted_labels = [int(result["label"].split("_")[-1]) for result in results]
    return {"predicted_label": predicted_labels}


predictions = test_dataset.map(predict, batched=True, batch_size=32)

# accuracy
true_labels = test_dataset["label"]
predicted_labels = predictions["predicted_label"]

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")

Device set to use cuda:0


Map:   0%|          | 0/1614 [00:00<?, ? examples/s]

Accuracy: 79.74%


# Results for Orientation (Accuracy)

* Masked LM
  * Original Language - Trained: 86.55 %
* Causal LM
  * Original Language - Untrained: 42.94%
  * Original Language - Trained: 66.48%
  * English - Untrained: 58.12%
  * English - Trained: 79.74%
